In [5]:
import functools
import numpy as np
import pandas as pd
import nltk
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim

In [6]:
#Hyper Params Do Not Change

from LSTM import PAD_INDEX, UNK_INDEX, PAD_TOKEN, UNK_TOKEN, \
MAX_LENGTH, BATCH_SIZE, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM, N_LAYERS, DROPOUT_RATE, LR, N_EPOCHS, UNIVERSAL_TAGS

from LSTM import Tokenizer, TaggedDataset, BiLSTMTagger, Trainer, initialize_weights, build_dataset, collate

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
#Data Processing
nltk.download('brown')
nltk.download('universal_tagset')
X, y = build_dataset()
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2)

tokenizer = Tokenizer()
tokenizer.train(X_train)

train_data = TaggedDataset(X_train, y_train, tokenizer, MAX_LENGTH)
valid_data = TaggedDataset(X_valid, y_valid, tokenizer, MAX_LENGTH)

collate = functools.partial(collate, pad_index=PAD_INDEX)

train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE, collate_fn=collate, shuffle=True)
valid_dataloader = torch.utils.data.DataLoader(valid_data, batch_size=BATCH_SIZE, collate_fn=collate)

In [8]:
#Create Necessities for Training
vocab_size = len(tokenizer.vocab)
model = BiLSTMTagger(vocab_size, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM, N_LAYERS, DROPOUT_RATE, PAD_INDEX)
model.apply(initialize_weights)

optimizer = optim.Adam(model.parameters(), lr=LR)
criterion = nn.CrossEntropyLoss()

model = model.to(device)
criterion = criterion.to(device)

In [ ]:
# Start training
best_valid_loss = float('inf')
best_valid_acc = -1
train_losses = []
train_accs = []
valid_losses = []
valid_accs = []

epoch_to_train_losses = {}
epoch_to_train_accs = {}
epoch_to_valid_losses = {}
epoch_to_valid_accs = {}

for epoch in range(N_EPOCHS):
    train_loss, train_acc = Trainer.train(train_dataloader, model, criterion, optimizer, device)
    valid_loss, valid_acc = Trainer.evaluate(valid_dataloader, model, criterion, device)

    train_losses.extend(train_loss)
    train_accs.extend(train_acc)
    valid_losses.extend(valid_loss)
    valid_accs.extend(valid_acc)

    epoch_to_train_losses[epoch] = train_loss
    epoch_to_train_accs[epoch] = train_acc
    epoch_to_valid_losses[epoch] = valid_loss
    epoch_to_valid_accs[epoch] = valid_acc

    epoch_train_loss = np.mean(train_loss)
    epoch_train_acc = np.mean(train_acc)
    epoch_valid_loss = np.mean(valid_loss)
    epoch_valid_acc = np.mean(valid_acc)

    # Save the model that achieves the smallest validation loss.
    if epoch_valid_loss < best_valid_loss:
        best_valid_loss = epoch_valid_loss
        torch.save(model.state_dict(), 'valid_loss_lstm.pt')
    
    if epoch_valid_acc > best_valid_acc:
        best_valid_acc = epoch_valid_acc
        torch.save(model.state_dict(), 'valid_acc_lstm.pt')


    print(f'epoch: {epoch+1}')
    print(f'train_loss: {epoch_train_loss:.3f}, train_acc: {epoch_train_acc:.3f}')
    print(f'valid_loss: {epoch_valid_loss:.3f}, valid_acc: {epoch_valid_acc:.3f}')


In [ ]:
epoch_to_train_losses = pd.DataFrame(epoch_to_train_losses)
epoch_to_train_accs = pd.DataFrame(epoch_to_train_accs)
epoch_to_valid_losses = pd.DataFrame(epoch_to_valid_losses)
epoch_to_valid_accs = pd.DataFrame(epoch_to_valid_accs)

epoch_to_train_losses.to_csv('epoch_to_train_losses.csv')
epoch_to_train_accs.to_csv('epoch_to_train_accs.csv')
epoch_to_valid_losses.to_csv('epoch_to_valid_losses.csv')
epoch_to_valid_accs.to_csv('epoch_to_valid_accs.csv')

In [ ]:
from google.colab import files

files.download('epoch_to_train_losses.csv')
files.download('epoch_to_train_accs.csv')
files.download('epoch_to_valid_losses.csv')
files.download('epoch_to_valid_accs.csv')
files.download('valid_loss_lstm.pt')
files.download('valid_acc_lstm.pt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>